# LSTM

다층 LSTM RNN을 입력 시퀀스에 적용합니다. 입력 시퀀스의 각 요소에 대해 각 계층은 다음 함수를 계산합니다:

$$
i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi})
$$

$$
f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf})
$$

$$
g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg})
$$

$$
o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho})
$$

$$
c_t = f_t \odot c_{t-1} + i_t \odot g_t
$$

$$
h_t = o_t \odot \tanh(c_t)
$$

여기서 \( h_t \)는 시간 \( t \)에서의 은닉 상태, \( c_t \)는 시간 \( t \)에서의 셀 상태, \( x_t \)는 시간 \( t \)에서의 입력, \( h_{t-1} \)은 시간 \( t-1 \)에서의 은닉 상태 또는 시간 0에서의 초기 은닉 상태, 그리고 \( i_t \), \( f_t \), \( g_t \), \( o_t \)는 각각 입력 게이트, 망각 게이트, 셀 게이트, 출력 게이트를 의미합니다. \( \sigma \)는 시그모이드 함수이며, \( \odot \)는 아다마르드 곱(요소별 곱)을 나타냅니다.

다층 LSTM에서 \( l \)번째 계층(\( l \geq 2 \))의 입력 \( x_t^{(l)} \)는 이전 계층의 은닉 상태 \( h_t^{(l-1)} \)에 드롭아웃 \( \delta_t^{(l-1)} \)을 곱한 것입니다. 여기서 각 \( \delta_t^{(l-1)} \)는 확률적으로 0이 되는 베르누이 확률 변수입니다.

만약 `proj_size > 0`이 지정되면, 투영이 있는 LSTM이 사용됩니다. 이는 LSTM 셀을 다음과 같이 변경합니다. 첫째, \( h_t \)의 차원이 `hidden_size`에서 `proj_size`로 변경됩니다(`W_{hi}`의 차원도 이에 맞게 변경됨). 둘째, 각 계층의 출력 은닉 상태는 학습 가능한 투영 행렬에 의해 곱해집니다: \( h_t = W_{hr} h_t \). 이로 인해 LSTM 네트워크의 출력 모양도 달라지게 됩니다. 자세한 내용은 [논문](https://arxiv.org/abs/1402.1128)을 참조하십시오.

## 파라미터

- **input_size**: 입력 \( x \)에서 기대되는 특징(feature)의 수
- **hidden_size**: 은닉 상태 \( h \)에서의 특징(feature)의 수
- **num_layers**: 순환 계층(recurrent layer)의 수. 예를 들어, `num_layers=2`로 설정하면 두 개의 LSTM을 쌓아 스택된 LSTM을 형성하며, 두 번째 LSTM은 첫 번째 LSTM의 출력을 받아 최종 결과를 계산합니다. 기본값: 1
- **bias**: False로 설정하면, 계층은 \( b_{ih} \)와 \( b_{hh} \) 편향 가중치를 사용하지 않습니다. 기본값: True
- **batch_first**: True로 설정하면 입력 및 출력 텐서를 (seq, batch, feature) 대신 (batch, seq, feature)로 제공합니다. 이는 은닉 상태 또는 셀 상태에는 적용되지 않습니다. 자세한 내용은 아래의 입력/출력 섹션을 참조하십시오. 기본값: False
- **dropout**: 0이 아닌 경우, 마지막 계층을 제외한 각 LSTM 계층의 출력에 드롭아웃 층을 추가하며, 드롭아웃 확률은 `dropout`과 같습니다. 기본값: 0
- **bidirectional**: True로 설정하면 양방향 LSTM이 됩니다. 기본값: False
- **proj_size**: 0보다 크면, 해당 크기의 투영을 가진 LSTM을 사용합니다. 기본값: 0


## Examples:

In [1]:
import torch
import torch.nn as nn

rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))